In [523]:
from openpyxl import *
import datetime

In [524]:
def handle_str(str1):
    '''
    '''
    #if there is missing or multiple data, leave it blank
    if str1 == '?' or str1 == '？' or str1 == None or ';' in str1:
        str1 = ''
    # if state is presumed, eliminate '[' first to get match
    elif '[' in str1:
        str1 = str1[1:len(str1)-1]
        global presumed
        presumed = True
    
    return str1

In [525]:
def match_standard(field, value, ws):
    
    # dict indicating standard column for different field
    standard_dict = dict(state = 18, city = 20, country = 22, venue = 24, date = 26)
    
    for row in range(2, 150):
        # if we find a match, get standard name
        if ws.cell(row = row, column = standard_dict[field]).value == handle_str(value):
            # match successful
            success = True
            # get standard value
            standard = ws.cell(row = row, column = standard_dict[field] + 1).value
            break
        # if we can not find, standard is None
        else:
            standard = None    
            
    return standard
            

In [526]:
def match_process(field, ws):
    '''match standard for city, state, country and venue element
    '''
    
    # create a dict of field as key and column number as value
    dict1 = dict(city = 1, state = 4, country = 7, venue = 10, date = 13)
    
    # loop over field
    for row in range(2, 1298):
        
        # did we find a match successfully?
        global success 
        success = False
        # a global variable to see if a value is presumed
        global presumed 
        presumed = False
        # get original value
        orig_value = ws.cell(row = row, column = dict1[field]).value
        # match a standard value
        standard = match_standard(field, orig_value, ws)
        
        # must consider if standard is None!
        if standard != None:
        # if presumed, add []
            if presumed:
                ws.cell(row = row, column = dict1[field] + 1).value = '[' + standard + ']'
            else:
                ws.cell(row = row, column = dict1[field] + 1).value = standard
           
        # if we can not find a standard, try fill its original value
        else:
            # if presumed, add []
            if presumed:
                ws.cell(row = row, column = dict1[field] + 1).value = '[' + handle_str(orig_value) + ']'
            else:
                ws.cell(row = row, column = dict1[field] + 1).value = handle_str(orig_value)
           
    
        # print state for which we cannot find a match
#         if success == False:
#             print (orig_value)

In [527]:
def convert_format(element):
    '''convert date format
    '''
    
    # initialize presumed
    global presumed
    presumed = False
    
    # if the whole date in a []
    if element[0] == '[' and element[len(element)-1] == ']':
        # elimiunate [], mark it as presumed
        element = handle_str(element)

    # if there is only year
    if element.count('/') == 0:
        year = handle_str(element)
        month = ''
        day = ''

    # if there are month and year
    elif element.count('/') == 1:
        year = element.strip().split('/')[1]
        month = '-' + element.strip().split('/')[0]
        day = ''

    # if there are day, month and year
    elif element.count('/') == 2:
        year = element.strip().split('/')[2]
        month = '-' + element.strip().split('/')[0]
        day = '-' + element.strip().split('/')[1]

    # if there is a date range
    elif element.count('/') > 2:
        # split two dates by '-'
        date_range = element.strip().split('-')
        # initialize return str
        range_str = ''
        # loop over two dates
        for date in date_range:
            range_str = range_str + convert_format(date) + '-'
            print(range_str)
    
        return range_str[:len(range_str)-1]
    
    return year + month + day

In [528]:
def match_date(ws):
    '''this method is to create a standard date list
    '''

#     # set up date style
#     style1 = styles.NamedStyle(name='custom_datetime1', number_format='DD/MM/YYYY HH:MM')
#     style2 = styles.NamedStyle(name='custom_datetime2', number_format='YYYY-MM-DD')
    
    # loop over date field
    for row in range(2,241):
        
#         # get dates
#         date = ws.cell(row = row, column = 14).value
#         ws.cell(row = row, column = 15).value = date
#         ws.cell(row = row, column = 16).value = date
        
#         # set date style to style1
#         ws.cell(row = row, column = 15).style = style1
#         print(ws.cell(row = row, column = 15).value)
        
#         # set date style to style2 
#         ws.cell(row = row, column = 16).style = style2
#         print(ws.cell(row = row, column = 16).value)
    
        # get date in string format
        date_str = ws.cell(row = row, column = 28).value
        # get date list
        if date_str != None:
            date = str(date_str).split(';')
            # initialize new date str we are gonna fill sheet with
            date_str1 = ''

            # loop over the dates in the list
            for element in date:

                new_date = convert_format(element)

                date_str1 = date_str1 + new_date + ' | '

            # get the final combination
            if presumed:
                ws.cell(row = row, column = 29).value = '[' + date_str1[:len(date_str1)-3] + ']'
            else:
                ws.cell(row = row, column = 29).value = date_str1[:len(date_str1)-3]
            
            print (ws.cell(row = row, column = 29).value)
            
    return ws

In [529]:
def main():
    
    wb = load_workbook\
             (r'/Users/zhiyuzhou/OneDrive/Penn/18_Summer/Work_on-campus/Script/Descriptive_matadata/uniq_element_list_178-179_4.3.19.xlsx', data_only=True)

    ws = wb["Sheet1"]
    
#     match_process('city', ws)
    
#     match_process('state', ws)
    
#     match_process('country', ws)
    
#     match_process('venue', ws)
    
    match_date(ws)
    
    print('match done...')
    
    # save workbook
    wb.save(r'/Users/zhiyuzhou/OneDrive/Penn/18_Summer/Work_on-campus/Script/Descriptive_matadata/uniq_element_list_178-179_4.3.19.xlsx')

if __name__ == '__main__':
    main()

[1927-1934]
[1930]
[1935]
[1939]
[194?]
[1940-1941]
[1943]
[1946]
[1948]
[1948-1949]
[1949]
[1957-1958]
[1964-1965]
1931
1937
1938
1948
1949
1950
1951
1952
1953
1956
1957
1957 | 1957-02-01 | 1957-02-07 | 1957-02-11 | 1957-02-14 | 1957-02-17
1936
[1940-1939]
[193x]
[1940]
[1947-8]
[1959-1958]
1948
1952
1974
19xx
[1945]
[1955-02-01]
[1943-02-18]
[1963-02-20]
[1942-03-11]
[1942-04-06]
[1950-04-19]
[1930-05]
[1943-05-21]
1962-[06]
[1950-07]
1964-[09]
1959-[09-11]
[1942-11-04]
[1942-11-13]
[1945-12]
[1935-12-06]
1935-[12]-[10]
[1925]
[1926]
[1928]
[1930]
[1930]]
[1931-1935]
[1932]
[1933-1934]
[1934]
[1935]
[1936-1937]
[1937]
[1937-1938]
[1938-1939]
[1939-1940]
[1941-1942]
[1944-1945]
[1944-1945]
[1945-1946]
[1961]
1943-01
[1938-1939]-01-04
[1942]-01-05
[1937]-01-06
1945-01-07 | 1945-03-11 | 1945-04-15
1949-01-08 | 1949-03-07
1946-01-10 | 1946-01-12
1949-01-13 | 1949-01-23
1945-01-14 | 1945-01-22 | 1945-01-29 | 1945-02-01
1950-01-15 | 1950-04-23
1954-01-15 | 1954-01-16
19xx-01-15
19xx-01-15 